# Converting Images to a Cloud Volume Image Layer

This module demonstrates how to convert a sequence of image files into a CLoudVolume image layer. The images can be in any standard format, including TIFF, PNG, JPEG, BMP, and many more. See the [scikit-image docs](https://scikit-image.org/docs/dev/api/skimage.io.html "skimage.io Documentation") for more information about file types.

In [1]:
# We need to add balsam and the modules it depends on to the Python search paths. 
import sys
sys.path.insert(0,'/soft/datascience/Balsam/0.3.5.1/env/lib/python3.6/site-packages/')
sys.path.insert(0,'/soft/datascience/Balsam/0.3.5.1/')

# We also need balsam and postgresql to be in the path. (Misha suggests this may not be necessary)
import os
os.environ['PATH'] ='/soft/datascience/Balsam/0.3.5.1/env/bin/:' + os.environ['PATH']
os.environ['PATH'] +=':/soft/datascience/PostgreSQL/9.6.12/bin/'
try:
    import balsam
except ImportError:
    print('Cannot find balsam, make sure balsam is installed or it is available in Python search paths')
    
os.environ["BALSAM_DB_PATH"]='/lus/theta-fs0/projects/connectomics_aesp/balsam_database/'

from balsam_helper import *

# Import widgets
from ipywidgets import interact, interactive
from ipywidgets import fixed, interact_manual 
from ipywidgets import Textarea, widgets, Layout, Accordion
from ipywidgets import VBox, HBox, Box, Text, BoundedIntText

env_preamble = '/lus/theta-fs0/projects/connectomics_aesp/software/HappyNeuron/macros_theta/theta_build_preamble.sh'

## Set up the Application

Before submitting image conversion jobs, make sure there is a Balsam application available to you. Here, we set up a HappyNeuron application and img2cv workflow. The application will run the img2cv executable installed with HappyNeuron.

In [2]:
add_app(
    'HappyNeuron_meshing',
    'python /home/kinnison/HappyNeuron/happyneuron/mesh/mesh_generator.py',  # 'mesh_generator',
    description='Create a 3D segmentation mesh.',
    envscript=env_preamble
)

Created new app


Application 15:
-----------------------
name:                           HappyNeuron_meshing
description:                    Create a 3D segmentation mesh.
executable:                     python /home/kinnison/HappyNeuron/happyneuron/mesh/mesh_generator.py
preprocess:                     
envscript:                      /lus/theta-fs0/projects/connectomics_aesp/software/HappyNeuron/macros_theta/theta_build_preamble.sh
postprocess:                    

## Set up the Workflow and Job

This is where job parameters will be set and added to a workflow in the application.

In [13]:
seg_path = '/lus/theta-fs0/projects/connectomics_aesp/pipeline_data/generate_mesh/precomputed/segmentation'

args = f'--labels {seg_path}'

job_id = add_job(
    'generate_mesh',  # Job Name
    'generate_mesh_rafcube',   # Workflow Name
    'HappyNeuron_meshing',
    description='Convert the Allen stack to a CloudVolume image layer.',
    args=args,
    num_nodes=2,
    ranks_per_node=4
)

## Run the Conversion

In [14]:
submit(project='connectomics_aesp',
   queue='debug-flat-quad',
   nodes=2,
   wall_minutes=20,
   wf_filter='generate_mesh_rafcube'
)

Submit OK: Qlaunch {   'command': '/lus/theta-fs0/projects/connectomics_aesp/balsam_database/qsubmit/qlaunch98.sh',
    'from_balsam': True,
    'id': 98,
    'job_mode': 'mpi',
    'nodes': 2,
    'prescheduled_only': False,
    'project': 'connectomics_aesp',
    'queue': 'debug-flat-quad',
    'scheduler_id': 379875,
    'state': 'submitted',
    'wall_minutes': 20,
    'wf_filter': 'generate_mesh_rafcube'}


In [16]:
def get_job_info(job_id='',show_output=False):
    """
    Prints verbose job info for a given job id.
    Parameters
    ----------
    job_id: str, Partial or full Balsam job id.
    """
    from balsam.launcher.dag import BalsamJob as Job
    jobs = Job.objects.all().filter(job_id__contains=job_id)
    if len(jobs) == 1:
        thejob = jobs[0]
        print(jobs[0])
        if show_output:
            output = f'{thejob.working_directory}/{thejob.name}.out'
            with open(output) as f:
                out = f.read()
            print(f'Output file {output} content:')
            print(out)
    elif len(jobs) == 0:
        print('No matching jobs')
    else:
        print(f'{len(jobs)} jobs matched, enter full id.')
        
get_job_info(job_id=job_id, show_output=True)

BalsamJob 3af7d6c9-b808-404f-ba6f-894b95329e61
----------------------------------------------
workflow:                       generate_mesh_rafcube
name:                           generate_mesh
description:                    Convert the Allen stack to a CloudVolume image layer.
lock:                           thetamom2:37025
parents:                        []
input_files:                    *
stage_in_url:                   
stage_out_files:                
stage_out_url:                  
wall_time_minutes:              1
num_nodes:                      2
coschedule_num_nodes:           0
ranks_per_node:                 4
cpu_affinity:                   depth
threads_per_rank:               1
threads_per_core:               1
node_packing_count:             1
environ_vars:                   KMP_BLOCKTIME=0:KMP_AFFINITY=granularity=fine,verbose,compact,1,0
application:                    HappyNeuron_meshing
args:                           --labels /lus/theta-fs0/projects/connectomics_